In [1]:
import sys
sys.path.append('../../')

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

import zipfile
import os
from pathlib import Path
import urllib.request


In [2]:
spark = SparkSession.builder \
    .appName("ner")\
    .master("local[1]")\
    .config("spark.driver.memory","8G")\
    .config("spark.driver.maxResultS1ize", "2G") \
    .config("spark.jars.packages", "JohnSnowLabs:spark-nlp:1.8.3")\
    .config("spark.kryoserializer.buffer.max", "500m")\
    .getOrCreate()

1. Download CoNLL2003 dataset
2. Save 3 files eng.train, eng.testa, eng.testa, into working dir ./

In [3]:
# Example how to download CoNLL 2003 Dataset
def download_conll2003_file(file):    
    if not Path(file).is_file():
        url = "https://raw.githubusercontent.com/patverga/torch-ner-nlp-from-scratch/master/data/conll2003/" + file
        urllib.request.urlretrieve(url, file)
        
download_conll2003_file("eng.train")
download_conll2003_file("eng.testa")
download_conll2003_file("eng.testb")

3 Download Glove word embeddings

In [4]:
file = "glove.6B.zip"
if not Path("glove.6B.zip").is_file():
    url = "http://nlp.stanford.edu/data/glove.6B.zip"
    print("Start downoading Glove Word Embeddings. It will take some time, please wait...")
    urllib.request.urlretrieve(url, "glove.6B.zip")
    print("Downloading finished")
    
if not Path("glove.6B.100d.txt").is_file():
    zip_ref = zipfile.ZipFile(file, 'r')
    zip_ref.extractall("./")
    zip_ref.close()


In [5]:
from pyspark.sql.types import *

class Annotation:
    def __init__(self, annotatorType, begin, end, result, metadata):
        self.annotatorType = annotatorType
        self.begin = begin
        self.end = end
        self.result = result
        self.metadata = metadata

        
annotation_schema = StructType([
    StructField("annotatorType", StringType()),
    StructField("begin", IntegerType(), False),
    StructField("end", IntegerType(), False),
    StructField("result", StringType()),
    StructField("metadata", MapType(StringType(), StringType()))
])
    


def readDataset(file, doc_column = "text", label_column = "label"):
    global spark
    
    result = []
    doc = ""
    labels = []

    with open(file) as f:
        for line in f:
            items = line.split(' ')
            word = items[0]
            if word == "-DOCSTART-":
                result.append((doc, labels))
                doc = ""
                labels = []
            elif len(items) <= 1:
                doc = doc + " \n"
            else:
                if len(doc) > 0:
                    doc = doc + " "

                begin = len(doc)
                doc = doc + word
                end = len(doc) - 1
                ner = items[3]
                labels.append(Annotation("named_entity", begin, end, ner, {}))

    if doc:
        result.append((doc, labels))
    
    global annotation_schema
    
    schema =  StructType([
      StructField(doc_column, StringType()),
      StructField(label_column, ArrayType(annotation_schema))
    ])
    
    
    return spark.createDataFrame(result, schema = schema)

In [6]:
import time


def get_pipeline():
    documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

    sentenceDetector = SentenceDetector()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")

    tokenizer = Tokenizer()\
      .setInputCols(["document"])\
      .setOutputCol("token")

    # download predefined model
    from sparknlp.pretrained import ResourceDownloader
    downloader = ResourceDownloader()
    posTagger = downloader.downloadModel(PerceptronModel, "pos_fast", "en")
    posTagger.setInputCols(["token", "document"])\
    .setOutputCol("pos")

    nerTagger = NerCrfApproach()\
      .setInputCols(["sentence", "token", "pos"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMinEpochs(1)\
      .setMaxEpochs(10)\
      .setLossEps(1e-3)\
      .setExternalFeatures("../../../src/test/resources/ner-corpus/dict.txt", ",")\
      .setEmbeddingsSource("glove.6B.100d.txt", 100, 2)\
      .setL2(1)\
      .setC0(1250000)\
      .setRandomSeed(100)\
      .setVerbose(2)
      
    pipeline = Pipeline(
        stages = [
        documentAssembler,
        sentenceDetector,
        tokenizer,
        posTagger,
        nerTagger
      ])
    
    return pipeline


def train_model(file):
    global spark
    
    print("Dataset Reading")
    
    start = time.time()
    dataset = readDataset(file)
    print("Done, {}\n".format(time.time() - start))

    print("Start fitting")
    pipeline = get_pipeline()

    return pipeline.fit(dataset)
 


In [7]:
from pyspark.sql.functions import col, udf, explode


def get_dataset_for_analysis(file, model):
    global spark
    
    print("Dataset Reading")
    
    start = time.time()
    dataset = readDataset(file)
    print("Done, {}\n".format(time.time() - start))
    
    predicted = model.transform(dataset)
    
    global annotation_schema
    
    zip_annotations = udf(
      lambda x, y: list(zip(x, y)),
      ArrayType(StructType([
          StructField("predicted", annotation_schema),
          StructField("label", annotation_schema)
      ]))
    )
    
    return predicted\
        .withColumn("result", zip_annotations("ner", "label"))\
        .select(explode("result").alias("result"))\
        .select(
            col("result.predicted").alias("predicted"), 
            col("result.label").alias("label")
        )
        
def printStat(label, correct, predicted, predictedCorrect):
    prec = predictedCorrect / predicted if predicted > 0 else 0
    rec = predictedCorrect / correct if correct > 0 else 0
    f1 = (2*prec*rec)/(prec + rec) if prec + rec > 0 else 0
    
    print("{}\t{}\t{}\t{}".format(label, prec, rec, f1))
        

def test_dataset(file, model, ignore_tokenize_misses=True):
    global spark
    
    started = time.time()

    df = readDataset(file)
    transformed = model.transform(df).select("label", "ner")

    labels = []
    predictedLabels = []

    for line in transformed.collect():
        label = line[0]
        ner = line[1]
    
        ner = {(a["begin"], a["end"]):a["result"] for a in ner}

        for a in label:
            key = (a["begin"], a["end"])

            label = a["result"].strip()
            predictedLabel = ner.get(key, "O").strip()
            
            if key not in ner and ignore_tokenize_misses:
                continue
                
            labels.append(label)
            predictedLabels.append(predictedLabel)
        

    correct = {}
    predicted = {}
    predictedCorrect = {}


    print(len(labels))

    for (lPredicted, lCorrect) in zip(predictedLabels, labels):
        correct[lCorrect] = correct.get(lCorrect, 0) + 1
        predicted[lPredicted] = predicted.get(lPredicted, 0) + 1

        if lCorrect == lPredicted:
            predictedCorrect[lPredicted] = predictedCorrect.get(lPredicted, 0) + 1

    correct = { key: correct[key] for key in correct.keys() if key != 'O'}
    predicted = { key: predicted[key] for key in predicted.keys() if key != 'O'}
    predictedCorrect = { key: predictedCorrect[key] for key in predictedCorrect.keys() if key != 'O'}

    tags = set(list(correct.keys()) + list(predicted.keys()))

    print("label\tprec\trec\tf1")
    totalCorrect = sum(correct.values())
    totalPredicted = sum(predicted.values())
    totalPredictedCorrect = sum(predictedCorrect.values())

    printStat("Total", totalCorrect, totalPredicted, totalPredictedCorrect)

    for label in tags:
        printStat(label, correct.get(label, 0), predicted.get(label, 0), predictedCorrect.get(label, 0))


In [8]:
import os.path

folder = '.'
train_file = os.path.join(folder, "eng.train")
test_file_a = os.path.join(folder, "eng.testa")
test_file_b = os.path.join(folder, "eng.testb")

In [9]:
model = train_model(train_file)

Dataset Reading
Done, 26.734439849853516

Start fitting


In [10]:
print("\nQuality on training data")
test_dataset(train_file, model)

print("\n\nQuality on validation data")
test_dataset(test_file_a, model)

print("\n\nQuality on test data")
test_dataset(test_file_b, model)


Quality on training data
200606
label	prec	rec	f1
Total	0.9766336988675411	0.9660400780280192	0.971308004338717
I-ORG	0.9629107034872002	0.9627166465135026	0.9628136652222111
B-MISC	0.8571428571428571	0.32432432432432434	0.4705882352941177
I-PER	0.9865995143448152	0.9892686446027595	0.9879322766570605
I-LOC	0.9798436354752016	0.9703605129445924	0.9750790177486021
B-ORG	1.0	1.0	1.0
B-LOC	1.0	0.6363636363636364	0.7777777777777778
I-MISC	0.9766388557806913	0.9138969440107071	0.9442267803641392


Quality on validation data
50584
label	prec	rec	f1
Total	0.9427812650893288	0.9141987592180733	0.9282700421940929
I-ORG	0.9060636182902585	0.8815280464216635	0.8936274509803923
B-MISC	0.0	0.0	0
I-PER	0.9727651393784044	0.969039259495691	0.9708986248800767
I-LOC	0.945357833655706	0.9372003835091084	0.9412614347616756
I-MISC	0.9213691026827012	0.7955271565495208	0.853836262323189


Quality on test data
45735
label	prec	rec	f1
Total	0.8912476722532588	0.8880504700643246	0.8896461986492349
I-ORG	0.84

In [11]:
df = get_dataset_for_analysis(test_file_a, model)
df.show()

Dataset Reading
Done, 2.7319753170013428

+--------------------+--------------------+
|           predicted|               label|
+--------------------+--------------------+
|[named_entity,3,9...|[named_entity,3,9...|
|[named_entity,11,...|[named_entity,11,...|
|[named_entity,13,...|[named_entity,13,...|
|[named_entity,28,...|[named_entity,28,...|
|[named_entity,33,...|[named_entity,33,...|
|[named_entity,38,...|[named_entity,38,...|
|[named_entity,41,...|[named_entity,41,...|
|[named_entity,45,...|[named_entity,45,...|
|[named_entity,51,...|[named_entity,51,...|
|[named_entity,59,...|[named_entity,59,...|
|[named_entity,67,...|[named_entity,67,...|
|[named_entity,71,...|[named_entity,71,...|
|[named_entity,78,...|[named_entity,78,...|
|[named_entity,91,...|[named_entity,91,...|
|[named_entity,96,...|[named_entity,96,...|
|[named_entity,103...|[named_entity,103...|
|[named_entity,115...|[named_entity,115...|
|[named_entity,120...|[named_entity,120...|
|[named_entity,128...|[named_entit

In [12]:
get_pipeline().write().overwrite().save("./crf_pipeline")
model.write().overwrite().save("./crf_model")

In [13]:
from pyspark.ml import PipelineModel, Pipeline

Pipeline.read().load("./crf_pipeline")
sameModel = PipelineModel.read().load("./crf_model")

In [14]:
print("\nQuality on training data")
test_dataset(train_file, sameModel)

print("\n\nQuality on validation data")
test_dataset(test_file_a, sameModel)

print("\n\nQuality on test data")
test_dataset(test_file_b, sameModel)


Quality on training data
200606
label	prec	rec	f1
Total	0.9766336988675411	0.9660400780280192	0.971308004338717
I-ORG	0.9629107034872002	0.9627166465135026	0.9628136652222111
B-MISC	0.8571428571428571	0.32432432432432434	0.4705882352941177
I-PER	0.9865995143448152	0.9892686446027595	0.9879322766570605
I-LOC	0.9798436354752016	0.9703605129445924	0.9750790177486021
B-ORG	1.0	1.0	1.0
B-LOC	1.0	0.6363636363636364	0.7777777777777778
I-MISC	0.9766388557806913	0.9138969440107071	0.9442267803641392


Quality on validation data
50584
label	prec	rec	f1
Total	0.9427812650893288	0.9141987592180733	0.9282700421940929
I-ORG	0.9060636182902585	0.8815280464216635	0.8936274509803923
B-MISC	0.0	0.0	0
I-PER	0.9727651393784044	0.969039259495691	0.9708986248800767
I-LOC	0.945357833655706	0.9372003835091084	0.9412614347616756
I-MISC	0.9213691026827012	0.7955271565495208	0.853836262323189


Quality on test data
45735
label	prec	rec	f1
Total	0.8912476722532588	0.8880504700643246	0.8896461986492349
I-ORG	0.84